In [ ]:
import pandas as pd
import  random
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
from datetime import datetime, date
from faker import Faker

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "HIST_ATUALIZACOES"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")


#DATA DA NOTA
data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado

#CRIAR O NUMERO DA NOTA FISCAL

try:
    with engine.connect() as conn:
            num_nf_no_bancodados = conn.execute(text("SELECT COUNT(NUMERONF) FROM NOTAFISCAL")).scalar()
            print(f"Numero da ultima nota fiscal do banco de dados: {num_nf_no_bancodados}")            
except Exception as e:
    print(f"Erro ao executar: {e}")

numeronf = num_nf_no_bancodados + 1

#tipo_dados_ITEMNOTA = {
#                'ID_NOTAFISCAL':  INTEGER,
#                'ID_PRODUTO':     INTEGER,
#                'QUANTIDADE':     INTEGER,
#                'PRECO_UNIT':     NUMERIC(18,2),
#                'TOTAL_ITEM':     NUMERIC(18,2),
#                'DATA_CRIACAO':   DATE,
#                'DATA_ATLZ':      DATE,
#                'VERSAO':         INTEGER,
#        }
#df_NOTAFISCAL = pd.DataFrame({
#                'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
#                'NUMERONF'        ?,
#                'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
#                'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
#                'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
#                'ID_LOJA':        df_todas_notas2['ID_LOJA'],
#                'ID_CFOP':        df_id_cfop['ID_CFOP'],
#                'CHAVE_ACESSO':   valores_unicos,
#                'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
#                'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
#                'VERSAO':         df_todas_notas2['VERSAO'],
#})

Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida
Numero da ultima nota fiscal do banco de dados: 0
